In [2]:
import yfinance as yf
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from gpflow.kernels import Matern32, Kernel
from gpflow.models import GPR
from gpflow import set_trainable
from sklearn.preprocessing import StandardScaler
import vectorbt as vbt
from datetime import datetime, timedelta
import ruptures as rpt
import warnings

warnings.filterwarnings("ignore", category=FutureWarning)

# Define the custom ChangePointKernel for Gaussian Process Regression
class ChangePointKernel(Kernel):
    def __init__(self, base_kernel, changepoints):
        super().__init__()
        self.base_kernel = base_kernel
        # changepoints: list of indices where a regime change is detected.
        self.changepoints = changepoints

    def get_region(self, X):
        cp = tf.constant(self.changepoints, dtype=X.dtype)
        regions = tf.searchsorted(cp, X[:, 0], side='right')
        return regions

    def K(self, X, X2=None):
        if X2 is None:
            X2 = X
        regions_X = self.get_region(X)
        regions_X2 = self.get_region(X2)
        regions_equal = tf.cast(tf.equal(tf.expand_dims(regions_X, 1), tf.expand_dims(regions_X2, 0)), X.dtype)
        base_cov = self.base_kernel.K(X, X2)
        return base_cov * regions_equal

    def K_diag(self, X):
        return self.base_kernel.K_diag(X)

def fetch_and_process_data(ticker, start_date, end_date):
    data = yf.download(ticker, start=start_date, end=end_date)
    if data.empty:
        raise ValueError(f"No data found for {ticker} between {start_date} and {end_date}")
    # Prefer "Adj Close" if available, otherwise "Close"
    if "Adj Close" in data.columns:
        price_series = data["Adj Close"].copy()
    else:
        price_series = data["Close"].copy()
    if isinstance(price_series, pd.DataFrame):
        price_series = price_series.squeeze()
    price_series.name = ticker
    returns = price_series.pct_change().dropna()
    scaler = StandardScaler()
    std_returns = scaler.fit_transform(returns.values.reshape(-1, 1))
    valid_index = returns.index
    return price_series.loc[valid_index], pd.Series(
        std_returns.flatten(), index=valid_index, name="Standardized Returns"
    )

def detect_changepoints(returns, lookback_window):
    # Use ruptures to detect changepoints using an RBF model.
    algo = rpt.Pelt(model="rbf").fit(returns.values)
    # Use the lookback_window as penalty; this is a simplified proxy for CPD LBW.
    result = algo.predict(pen=lookback_window)
    return result

def generate_signals(returns, changepoints, train_ratio, epochs, 
                     dropout_rate, lstm_hidden_units, learning_rate, batch_size):
    # Use time index as a feature.
    X = np.arange(len(returns), dtype=np.float64).reshape(-1, 1)
    y = returns.values.reshape(-1, 1)
    # Use Gaussian Process Regression with the ChangePointKernel for trend extraction.
    base_kernel = Matern32()
    kernel = ChangePointKernel(base_kernel, changepoints)
    gpr = GPR(data=(X, y), kernel=kernel)
    set_trainable(gpr.likelihood.variance, False)
    trend = gpr.predict_f(X)[0].numpy().flatten()
    # Combine returns and trend to form a feature set.
    features = np.hstack([returns.values.reshape(-1, 1), trend.reshape(-1, 1)])
    split_index = int(len(features) * train_ratio)
    train_features = features[:split_index]
    train_labels = returns.values[:split_index]
    # Reshape features for LSTM input.
    train_features = train_features.reshape((train_features.shape[0], 1, train_features.shape[1]))
    # Build the LSTM model.
    model = Sequential([
        LSTM(lstm_hidden_units, return_sequences=True,
             input_shape=(train_features.shape[1], train_features.shape[2])),
        Dropout(dropout_rate),
        LSTM(int(lstm_hidden_units / 2)),
        Dense(1, activation="tanh")
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), loss="mse")
    model.fit(train_features, train_labels, epochs=epochs, batch_size=batch_size, verbose=0)
    # Predict signals for all features.
    all_features = features.reshape((features.shape[0], 1, features.shape[1]))
    predicted_signals = model.predict(all_features, verbose=0).flatten()
    positions = np.clip(predicted_signals, -1, 1)
    return positions

def backtest_strategy(price_series, positions):
    if isinstance(price_series, pd.DataFrame):
        price_series = price_series.squeeze()
    positions = pd.Series(positions, index=price_series.index, name=price_series.name)
    entries = positions > 0
    exits = positions < 0
    pf = vbt.Portfolio.from_signals(
        close=price_series,
        entries=entries,
        exits=exits,
        size=np.abs(positions),
        freq="1D",
        fees=0.001,
    )
    return pf

def compute_trade_statistics(pf, price_series):
    # Extract trade records from the portfolio
    trades = pf.trades.records
    if trades.empty:
        win_rate = np.nan
        avg_win = np.nan
        avg_loss = np.nan
    else:
        # Assuming 'pnl' column exists representing profit (can be negative).
        wins = trades[trades.pnl > 0]
        losses = trades[trades.pnl < 0]
        win_rate = len(wins) / len(trades) if len(trades) > 0 else np.nan
        avg_win = wins.pnl.mean() if not wins.empty else np.nan
        avg_loss = losses.pnl.mean() if not losses.empty else np.nan
    return win_rate, avg_win, avg_loss

def main():
    # Fixed hyperparameters.
    params = {
        'batch_size': 128,
        'dropout_rate': 0.2,
        'learning_rate': 0.001,
        'lookback_window': 21,
        'lstm_hidden_units': 40,
        'epochs': 50,
        'train_ratio': 0.8
    }
    
    # Updated dictionary of top cryptocurrencies by year with -USD suffix
    top_cryptos = {
        "2019": ["BTC-USD", "ETH-USD", "XRP-USD", "BCH-USD", "EOS-USD", "LTC-USD", "XLM-USD", "ADA-USD", "TRX-USD", "BSV-USD"],
        "2020": ["BTC-USD", "ETH-USD", "XRP-USD", "BCH-USD", "LTC-USD", "EOS-USD", "BNB-USD", "BSV-USD", "ADA-USD", "XTZ-USD"],
        "2021": ["BTC-USD", "ETH-USD", "XRP-USD", "LTC-USD", "BCH-USD", "ADA-USD", "DOT-USD", "LINK-USD", "BNB-USD", "XLM-USD"],
        "2022": ["BTC-USD", "ETH-USD", "BNB-USD", "SOL-USD", "ADA-USD", "XRP-USD", "DOT-USD", "LUNA-USD", "AVAX-USD", "DOGE-USD"],
        "2023": ["BTC-USD", "ETH-USD", "BNB-USD", "XRP-USD", "ADA-USD", "DOGE-USD", "MATIC-USD", "DOT-USD", "LTC-USD", "SHIB-USD"],
        "2024": ["BTC-USD", "ETH-USD", "BNB-USD", "XRP-USD", "ADA-USD", "DOGE-USD", "SOL-USD", "DOT-USD", "LTC-USD", "AVAX-USD"],
        "2025": ["BTC-USD", "ETH-USD", "BNB-USD", "XRP-USD", "ADA-USD", "DOGE-USD", "SOL-USD", "DOT-USD", "LTC-USD", "AVAX-USD"]
    }

    # Define periods (one per year)
    periods = []
    for year in range(2019, 2025):
        period_label = str(year)
        start_date = f"{year}-01-01"
        end_date = f"{year}-12-31"
        periods.append((period_label, start_date, end_date))
    # YTD 2025 using current date as provided
    periods.append(("2025_YTD", "2025-01-01", "2025-03-04"))

    results = []

    # Loop over each period and backtest the respective top cryptocurrencies for that year
    for period_label, start_date, end_date in periods:
        year = period_label.split("_")[0]  # Extract year from period label
        cryptos_for_year = top_cryptos.get(year, [])
        
        for crypto in cryptos_for_year:
            print(f"Processing {crypto} for period {period_label} ({start_date} to {end_date})")
            try:
                price_series, returns = fetch_and_process_data(crypto, start_date, end_date)
                if len(returns) < params['lookback_window']:
                    print(f"Not enough data for {crypto} in period {period_label}. Skipping.")
                    continue
                
                changepoints = detect_changepoints(returns, lookback_window=params['lookback_window'])
                positions = generate_signals(
                    returns, changepoints,
                    train_ratio=params['train_ratio'],
                    epochs=params['epochs'],
                    dropout_rate=params['dropout_rate'],
                    lstm_hidden_units=params['lstm_hidden_units'],
                    learning_rate=params['learning_rate'],
                    batch_size=params['batch_size']
                )
                portfolio = backtest_strategy(price_series, positions)
                
                result = {
                    "crypto": crypto,
                    "period": period_label,
                    # "start_date": start_date,
                    # "end_date": end_date,
                    **{key: str(value) if pd.isna(value) else value for key, value in portfolio.stats().items()}
                }
                
                results.append(result)
            
            except Exception as e:
                print(f"Error processing {crypto} for period {period_label}: {e}")

    # Save all summary results into a DataFrame.
    results_df = pd.DataFrame(results)
    print("\nBacktest Results:")
    print(results_df)
    # Save the DataFrame to a CSV file.
    results_df.to_csv("Backtested/crypto_backtest_results_by_top10year.csv", index=False)

if __name__ == "__main__":
    main()

Processing BTC-USD for period 2019 (2019-01-01 to 2019-12-31)
YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed
/opt/homebrew/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
[*********************100%***********************]  1 of 1 completed

Processing ETH-USD for period 2019 (2019-01-01 to 2019-12-31)



/opt/homebrew/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
[*********************100%***********************]  1 of 1 completed

Processing XRP-USD for period 2019 (2019-01-01 to 2019-12-31)


...

Processing LTC-USD for period 2025_YTD (2025-01-01 to 2025-03-04)



/opt/homebrew/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
[*********************100%***********************]  1 of 1 completed

Processing AVAX-USD for period 2025_YTD (2025-01-01 to 2025-03-04)



/opt/homebrew/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)



Backtest Results:
      crypto    period      Start        End   Period  Start Value  \
0    BTC-USD      2019 2019-01-02 2019-12-30 363 days        100.0   
1    ETH-USD      2019 2019-01-02 2019-12-30 363 days        100.0   
2    XRP-USD      2019 2019-01-02 2019-12-30 363 days        100.0   
3    BCH-USD      2019 2019-01-02 2019-12-30 363 days        100.0   
4    EOS-USD      2019 2019-01-02 2019-12-30 363 days        100.0   
..       ...       ...        ...        ...      ...          ...   
65  DOGE-USD  2025_YTD 2025-01-02 2025-03-03  61 days        100.0   
66   SOL-USD  2025_YTD 2025-01-02 2025-03-03  61 days        100.0   
67   DOT-USD  2025_YTD 2025-01-02 2025-03-03  61 days        100.0   
68   LTC-USD  2025_YTD 2025-01-02 2025-03-03  61 days        100.0   
69  AVAX-USD  2025_YTD 2025-01-02 2025-03-03  61 days        100.0   

     End Value  Total Return [%]  Benchmark Return [%]  \
0   177.048491         77.048491             84.941362   
1   102.227605          

In [3]:
backtest_df = pd.read_csv("Backtested/crypto_backtest_results_by_top10year.csv")

In [4]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
backtest_df

,crypto,period,Start,End,Period,Start Value,End Value,Total Return [%],Benchmark Return [%],Max Gross Exposure [%],Total Fees Paid,Max Drawdown [%],Max Drawdown Duration,Total Trades,Total Closed Trades,Total Open Trades,Open Trade PnL,Win Rate [%],Best Trade [%],Worst Trade [%],Avg Winning Trade [%],Avg Losing Trade [%],Avg Winning Trade Duration,Avg Losing Trade Duration,Profit Factor,Expectancy,Sharpe Ratio,Calmar Ratio,Omega Ratio,Sortino Ratio
0,BTC-USD,2019,2019-01-02,2019-12-30,363 days,100.0,177.048491,77.048491,84.941362,100.000000,2.336236e+01,22.717278,89 days,81,81,0,0.000000,33.333333,20.367978,-13.196963,6.456962,-1.863916,3 days 08:53:20,1 days 05:46:40,1.505396,9.512159e-01,1.434209,3.416193,1.410626,2.441116
1,ETH-USD,2019,2019-01-02,2019-12-30,363 days,100.0,102.227605,2.227605,-14.456327,100.000000,1.176495e+01,22.889084,187 days,90,90,0,0.000000,31.111111,23.611161,-9.071376,6.999833,-2.242104,3 days 08:34:17.142857142,1 days 07:44:30.967741935,1.020906,2.475117e-02,0.259388,0.097864,1.067236,0.402717
2,XRP-USD,2019,2019-01-02,2019-12-30,363 days,100.0,99.966257,-0.033743,-48.173852,0.427028,2.611603e-02,0.100628,229 days,94,94,0,0.000000,25.531915,28.723192,-9.132107,5.631823,-2.255973,3 days 07:00:00,1 days 06:10:17.142857142,0.864431,-3.589729e-04,-0.282347,-0.337174,0.910778,-0.530683
3,BCH-USD,2019,2019-01-02,2019-12-30,363 days,100.0,130.980123,30.980123,21.869270,100.000000,1.534779e+01,40.001324,229 days,94,94,0,0.000000,35.106383,70.684594,-10.840778,8.003329,-2.991709,2 days 18:10:54.545454545,1 days 03:32:27.540983606,1.185893,3.295758e-01,0.742825,0.779350,1.233668,1.523975
4,EOS-USD,2019,2019-01-02,2019-12-30,363 days,100.0,97.804466,-2.195534,-8.383521,8.300491,3.301568e-01,2.405278,228 days,95,95,0,0.000000,27.368421,35.625649,-15.128080,7.943998,-2.790933,3 days 00:55:23.076923076,1 days 09:02:36.521739130,0.505354,-2.311088e-02,-1.418363,-0.917771,0.668874,-1.690139
5,LTC-USD,2019,2019-01-02,2019-12-30,363 days,100.0,127.253562,27.253562,27.857105,100.000000,4.970507e+00,8.778422,184 days,87,87,0,0.000000,39.080460,51.882708,-10.483983,7.750090,-2.613998,2 days 21:52:56.470588235,1 days 03:37:21.509433962,1.698094,3.132593e-01,1.192216,3.123871,1.422128,2.417943
6,XLM-USD,2019,2019-01-02,2019-12-30,363 days,100.0,100.003663,0.003663,-61.490309,0.112706,6.306489e-03,0.016730,211 days,88,88,0,0.000000,30.681818,40.713323,-7.289124,7.616060,-2.338136,3 days 06:13:20,1 days 09:26:33.442622950,1.064545,4.162777e-05,0.132245,0.220169,1.036339,0.217167
7,ADA-USD,2019,2019-01-02,2019-12-30,363 days,100.0,100.006137,0.006137,-25.772238,0.089500,5.581706e-03,0.017040,201 days,95,95,0,0.000000,27.368421,20.572796,-8.426702,6.475758,-2.810021,3 days 00:55:23.076923076,1 days 10:05:13.043478260,1.107042,6.459824e-05,0.207977,0.362127,1.057557,0.356893
8,TRX-USD,2019,2019-01-02,2019-12-30,363 days,100.0,100.004606,0.004606,-33.735540,0.032220,2.013308e-03,0.007662,148 days,93,93,0,0.000000,32.258065,27.276977,-9.776020,6.347696,-2.663376,2 days 20:48:00,1 days 08:45:42.857142857,1.237268,4.952460e-05,0.414984,0.604406,1.116641,0.642543
9,BSV-USD,2019,2019-01-02,2019-12-30,363 days,100.0,201.552608,101.552608,2.205586,100.000000,5.357871e+00,14.557453,145 days,94,94,0,0.000000,29.787234,73.829293,-16.731618,9.608518,-3.180943,2 days 17:08:34.285714285,1 days 04:43:38.181818181,2.598470,1.080347e+00,1.245709,7.029556,2.105766,5.305264


In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from gpflow.kernels import Matern32, Kernel
from gpflow.models import GPR
from gpflow import set_trainable
from sklearn.preprocessing import StandardScaler
import vectorbt as vbt
from datetime import datetime, timedelta
import ruptures as rpt
import warnings

warnings.filterwarnings("ignore", category=FutureWarning)

# Define the custom ChangePointKernel for Gaussian Process Regression
class ChangePointKernel(Kernel):
    def __init__(self, base_kernel, changepoints):
        super().__init__()
        self.base_kernel = base_kernel
        # changepoints: list of indices where a regime change is detected.
        self.changepoints = changepoints

    def get_region(self, X):
        cp = tf.constant(self.changepoints, dtype=X.dtype)
        regions = tf.searchsorted(cp, X[:, 0], side='right')
        return regions

    def K(self, X, X2=None):
        if X2 is None:
            X2 = X
        regions_X = self.get_region(X)
        regions_X2 = self.get_region(X2)
        regions_equal = tf.cast(tf.equal(tf.expand_dims(regions_X, 1), tf.expand_dims(regions_X2, 0)), X.dtype)
        base_cov = self.base_kernel.K(X, X2)
        return base_cov * regions_equal

    def K_diag(self, X):
        return self.base_kernel.K_diag(X)

def fetch_and_process_data(tickers, start_date, end_date):
    """Fetch data for multiple tickers and return price and returns DataFrames."""
    price_data = {}
    returns_data = {}
    
    for ticker in tickers:
        try:
            data = yf.download(ticker, start=start_date, end=end_date)
            if data.empty:
                print(f"No data found for {ticker} between {start_date} and {end_date}")
                continue
                
            # Prefer "Adj Close" if available, otherwise "Close"
            if "Adj Close" in data.columns:
                price_series = data["Adj Close"].copy()
            else:
                price_series = data["Close"].copy()
                
            if isinstance(price_series, pd.DataFrame):
                price_series = price_series.squeeze()
                
            price_series.name = ticker
            returns = price_series.pct_change().dropna()
            
            # Standardize returns
            scaler = StandardScaler()
            std_returns = scaler.fit_transform(returns.values.reshape(-1, 1))
            valid_index = returns.index
            
            price_data[ticker] = price_series.loc[valid_index]
            returns_data[ticker] = pd.Series(
                std_returns.flatten(), index=valid_index, name=f"{ticker}_stdret"
            )
        except Exception as e:
            print(f"Error fetching {ticker}: {e}")
    
    # Convert to DataFrames
    prices_df = pd.DataFrame({ticker: series for ticker, series in price_data.items()})
    returns_df = pd.DataFrame({ticker: series for ticker, series in returns_data.items()})
    
    return prices_df, returns_df

def detect_changepoints(returns, lookback_window):
    """Detect changepoints in return series."""
    # Use ruptures to detect changepoints using an RBF model.
    algo = rpt.Pelt(model="rbf").fit(returns.values)
    # Use the lookback_window as penalty; this is a simplified proxy for CPD LBW.
    result = algo.predict(pen=lookback_window)
    return result

def generate_signals(returns, changepoints, train_ratio, epochs, 
                     dropout_rate, lstm_hidden_units, learning_rate, batch_size):
    """Generate trading signals using GPR+LSTM model."""
    # Use time index as a feature.
    X = np.arange(len(returns), dtype=np.float64).reshape(-1, 1)
    y = returns.values.reshape(-1, 1)
    
    # Use Gaussian Process Regression with the ChangePointKernel for trend extraction.
    base_kernel = Matern32()
    kernel = ChangePointKernel(base_kernel, changepoints)
    gpr = GPR(data=(X, y), kernel=kernel)
    set_trainable(gpr.likelihood.variance, False)
    trend = gpr.predict_f(X)[0].numpy().flatten()
    
    # Combine returns and trend to form a feature set.
    features = np.hstack([returns.values.reshape(-1, 1), trend.reshape(-1, 1)])
    split_index = int(len(features) * train_ratio)
    train_features = features[:split_index]
    train_labels = returns.values[:split_index]
    
    # Reshape features for LSTM input.
    train_features = train_features.reshape((train_features.shape[0], 1, train_features.shape[1]))
    
    # Build the LSTM model.
    model = Sequential([
        LSTM(lstm_hidden_units, return_sequences=True,
             input_shape=(train_features.shape[1], train_features.shape[2])),
        Dropout(dropout_rate),
        LSTM(int(lstm_hidden_units / 2)),
        Dense(1, activation="tanh")
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), loss="mse")
    model.fit(train_features, train_labels, epochs=epochs, batch_size=batch_size, verbose=0)
    
    # Predict signals for all features.
    all_features = features.reshape((features.shape[0], 1, features.shape[1]))
    predicted_signals = model.predict(all_features, verbose=0).flatten()
    
    # Convert signals to binary (-1 or 1) for direction only
    # For equal weighting, we only care about direction (long/short)
    positions = np.sign(predicted_signals)
    positions[positions == 0] = 1  # Convert any zeros to 1 (long)
    
    return positions

def backtest_strategy_single_asset(price_series, positions):
    """Backtest strategy for a single asset."""
    if isinstance(price_series, pd.DataFrame):
        price_series = price_series.squeeze()
        
    positions = pd.Series(positions, index=price_series.index, name=price_series.name)
    entries = positions > 0
    exits = positions < 0
    
    pf = vbt.Portfolio.from_signals(
        close=price_series,
        entries=entries,
        exits=exits,
        size=1.0,  # Always use full size
        freq="1D",
        fees=0.001,
    )
    return pf

def backtest_equal_weight_portfolio(prices_df, signals_dict):
    """Backtest a portfolio with equal weights across assets."""
    # Create a dictionary to store individual asset performance
    asset_results = {}
    
    for ticker in prices_df.columns:
        if ticker in signals_dict:
            # Get price series and signals for this ticker
            price_series = prices_df[ticker]
            positions = signals_dict[ticker]
            
            # Make sure positions align with price index
            positions_series = pd.Series(positions, index=price_series.index[:len(positions)])
            
            # Backtest the individual asset
            pf = backtest_strategy_single_asset(price_series, positions_series)
            asset_results[ticker] = pf
    
    # Calculate the equal weight allocation (1/n for each asset)
    n_assets = len(asset_results)
    if n_assets == 0:
        return None, {}
        
    weight_per_asset = 1.0 / n_assets
    
    # Create a combined DataFrame for returns
    combined_returns = pd.DataFrame()
    
    for ticker, pf in asset_results.items():
        returns = pf.returns()
        if not returns.empty:
            combined_returns[ticker] = returns
    
    # Calculate equal-weighted portfolio returns
    if not combined_returns.empty:
        # Replace NaN with 0 for calculation
        combined_returns.fillna(0, inplace=True)
        portfolio_returns = (combined_returns * weight_per_asset).sum(axis=1)
        
        # Calculate cumulative returns
        portfolio_cumulative = (1 + portfolio_returns).cumprod()
        
        # For simplicity, create a mock portfolio object with key stats
        portfolio_stats = {
            "total_return": portfolio_cumulative.iloc[-1] - 1 if len(portfolio_cumulative) > 0 else 0,
            "sharpe_ratio": portfolio_returns.mean() / portfolio_returns.std() * np.sqrt(252) if len(portfolio_returns) > 0 else 0,
            "max_drawdown": (portfolio_cumulative / portfolio_cumulative.cummax() - 1).min() if len(portfolio_cumulative) > 0 else 0,
            "win_rate": np.sum(portfolio_returns > 0) / len(portfolio_returns) if len(portfolio_returns) > 0 else 0,
        }
        
        # Create mock portfolio object
        mock_portfolio = type('obj', (object,), {
            'stats': lambda: portfolio_stats
        })
        
        return mock_portfolio, asset_results
    else:
        return None, {}

def main():
    # Fixed hyperparameters.
    params = {
        'batch_size': 128,
        'dropout_rate': 0.2,
        'learning_rate': 0.001,
        'lookback_window': 21,
        'lstm_hidden_units': 40,
        'epochs': 50,
        'train_ratio': 0.8
    }
    
    # Updated dictionary of top cryptocurrencies by year with -USD suffix
    top_cryptos = {
        "2019": ["BTC-USD", "ETH-USD", "XRP-USD", "BCH-USD", "EOS-USD", "LTC-USD", "XLM-USD", "ADA-USD", "TRX-USD", "BSV-USD"],
        "2020": ["BTC-USD", "ETH-USD", "XRP-USD", "BCH-USD", "LTC-USD", "EOS-USD", "BNB-USD", "BSV-USD", "ADA-USD", "XTZ-USD"],
        "2021": ["BTC-USD", "ETH-USD", "XRP-USD", "LTC-USD", "BCH-USD", "ADA-USD", "DOT-USD", "LINK-USD", "BNB-USD", "XLM-USD"],
        "2022": ["BTC-USD", "ETH-USD", "BNB-USD", "SOL-USD", "ADA-USD", "XRP-USD", "DOT-USD", "LUNA-USD", "AVAX-USD", "DOGE-USD"],
        "2023": ["BTC-USD", "ETH-USD", "BNB-USD", "XRP-USD", "ADA-USD", "DOGE-USD", "MATIC-USD", "DOT-USD", "LTC-USD", "SHIB-USD"],
        "2024": ["BTC-USD", "ETH-USD", "BNB-USD", "XRP-USD", "ADA-USD", "DOGE-USD", "SOL-USD", "DOT-USD", "LTC-USD", "AVAX-USD"],
        "2025": ["BTC-USD", "ETH-USD", "BNB-USD", "XRP-USD", "ADA-USD", "DOGE-USD", "SOL-USD", "DOT-USD", "LTC-USD", "AVAX-USD"]
    }

    # Define periods (one per year)
    periods = []
    for year in range(2019, 2025):
        period_label = str(year)
        start_date = f"{year}-01-01"
        end_date = f"{year}-12-31"
        periods.append((period_label, start_date, end_date))
    # YTD 2025 using current date
    periods.append(("2025_YTD", "2025-01-01", "2025-03-07"))

    results = []

    # Loop over each period and backtest the respective top cryptocurrencies for that year
    for period_label, start_date, end_date in periods:
        year = period_label.split("_")[0]  # Extract year from period label
        cryptos_for_year = top_cryptos.get(year, [])
        
        print(f"\nProcessing period {period_label} ({start_date} to {end_date})")
        print(f"Cryptocurrencies: {', '.join(cryptos_for_year)}")
        
        # Fetch all data for the period
        prices_df, returns_df = fetch_and_process_data(cryptos_for_year, start_date, end_date)
        
        # Generate signals for each cryptocurrency
        signals_dict = {}
        
        for crypto in cryptos_for_year:
            if crypto in returns_df.columns:
                try:
                    returns = returns_df[crypto]
                    if len(returns) < params['lookback_window']:
                        print(f"Not enough data for {crypto} in period {period_label}. Skipping.")
                        continue
                    
                    changepoints = detect_changepoints(returns, lookback_window=params['lookback_window'])
                    positions = generate_signals(
                        returns, changepoints,
                        train_ratio=params['train_ratio'],
                        epochs=params['epochs'],
                        dropout_rate=params['dropout_rate'],
                        lstm_hidden_units=params['lstm_hidden_units'],
                        learning_rate=params['learning_rate'],
                        batch_size=params['batch_size']
                    )
                    
                    signals_dict[crypto] = positions
                except Exception as e:
                    print(f"Error generating signals for {crypto}: {e}")
        
        # Backtest with equal weight allocation
        portfolio, asset_portfolios = backtest_equal_weight_portfolio(prices_df, signals_dict)
        
        if portfolio:
            # Record main portfolio results
            portfolio_stats = portfolio.stats()
            result = {
                "period": period_label,
                "strategy": "Equal Weight Portfolio",
                "num_assets": len(signals_dict),
                **{key: str(value) if pd.isna(value) else value for key, value in portfolio_stats.items()}
            }
            results.append(result)
            
            # Also record individual asset results
            for crypto, pf in asset_portfolios.items():
                asset_stats = pf.stats()
                asset_result = {
                    "period": period_label,
                    "crypto": crypto,
                    "strategy": "Individual Asset",
                    **{key: str(value) if pd.isna(value) else value for key, value in asset_stats.items()}
                }
                results.append(asset_result)

    # Save all summary results into a DataFrame.
    results_df = pd.DataFrame(results)
    print("\nBacktest Results:")
    print(results_df)
    
    # Save the DataFrame to a CSV file.
    results_df.to_csv("Backtested/equal_weight_7.csv", index=False)
    
    # Calculate and display yearly performance comparison
    portfolio_results = results_df[results_df["strategy"] == "Equal Weight Portfolio"]
    if not portfolio_results.empty:
        print("\nEqual Weight Portfolio Performance by Year:")
        yearly_perf = portfolio_results[["period", "total_return", "sharpe_ratio", "max_drawdown"]]
        print(yearly_perf)

if __name__ == "__main__":
    main()


Processing period 2019 (2019-01-01 to 2019-12-31)
Cryptocurrencies: BTC-USD, ETH-USD, XRP-USD, BCH-USD, EOS-USD, LTC-USD, XLM-USD, ADA-USD, TRX-USD, BSV-USD


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
/opt/homebrew/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
[********************


Processing period 2020 (2020-01-01 to 2020-12-31)
Cryptocurrencies: BTC-USD, ETH-USD, XRP-USD, BCH-USD, LTC-USD, EOS-USD, BNB-USD, BSV-USD, ADA-USD, XTZ-USD



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
/opt/homebrew/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
[*********************100%***********************]  1 of 1 completed


Processing period 2021 (2021-01-01 to 2021-12-31)
Cryptocurrencies: BTC-USD, ETH-USD, XRP-USD, LTC-USD, BCH-USD, ADA-USD, DOT-USD, LINK-USD, BNB-USD, XLM-USD


...


Processing period 2024 (2024-01-01 to 2024-12-31)
Cryptocurrencies: BTC-USD, ETH-USD, BNB-USD, XRP-USD, ADA-USD, DOGE-USD, SOL-USD, DOT-USD, LTC-USD, AVAX-USD



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
/opt/homebrew/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*******************


Processing period 2025_YTD (2025-01-01 to 2025-03-07)
Cryptocurrencies: BTC-USD, ETH-USD, BNB-USD, XRP-USD, ADA-USD, DOGE-USD, SOL-USD, DOT-USD, LTC-USD, AVAX-USD



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
/opt/homebrew/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)



Backtest Results:
      period                strategy  num_assets  total_return  sharpe_ratio  \
0       2019  Equal Weight Portfolio        10.0      0.436882      1.354863   
1       2019        Individual Asset         NaN           NaN           NaN   
2       2019        Individual Asset         NaN           NaN           NaN   
3       2019        Individual Asset         NaN           NaN           NaN   
4       2019        Individual Asset         NaN           NaN           NaN   
5       2019        Individual Asset         NaN           NaN           NaN   
6       2019        Individual Asset         NaN           NaN           NaN   
7       2019        Individual Asset         NaN           NaN           NaN   
8       2019        Individual Asset         NaN           NaN           NaN   
9       2019        Individual Asset         NaN           NaN           NaN   
10      2019        Individual Asset         NaN           NaN           NaN   
11      2020  Equal W

In [6]:
backtest_eqw_df = pd.read_csv("Backtested/equal_weight_7.csv")

In [7]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
backtest_eqw_df

,period,strategy,num_assets,total_return,sharpe_ratio,max_drawdown,win_rate,crypto,Start,End,Period,Start Value,End Value,Total Return [%],Benchmark Return [%],Max Gross Exposure [%],Total Fees Paid,Max Drawdown [%],Max Drawdown Duration,Total Trades,Total Closed Trades,Total Open Trades,Open Trade PnL,Win Rate [%],Best Trade [%],Worst Trade [%],Avg Winning Trade [%],Avg Losing Trade [%],Avg Winning Trade Duration,Avg Losing Trade Duration,Profit Factor,Expectancy,Sharpe Ratio,Calmar Ratio,Omega Ratio,Sortino Ratio
0,2019,Equal Weight Portfolio,10.0,0.436882,1.354863,-0.110291,0.325069,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019,Individual Asset,NaN,NaN,NaN,NaN,NaN,BTC-USD,2019-01-02,2019-12-30,363 days,100.0,184.526728,84.526728,84.941362,100.000000,2.440565e+01,22.717278,89 days 00:00:00,81.0,81.0,0.0,0.000000,33.333333,20.367978,-13.196963,6.456962,-1.863916,3 days 08:53:20,1 days 05:46:40,1.533479,1.043540e+00,1.512869,3.748276,1.434426,2.593118
2,2019,Individual Asset,NaN,NaN,NaN,NaN,NaN,ETH-USD,2019-01-02,2019-12-30,363 days,100.0,172.180136,72.180136,-14.456327,100.000000,2.519808e+01,24.640693,84 days 00:00:00,89.0,89.0,0.0,0.000000,31.460674,23.611161,-9.071376,6.937629,-2.095498,3 days 06:51:25.714285714,1 days 06:41:18.688524590,1.391025,8.110128e-01,1.269196,2.950257,1.325192,2.194654
3,2019,Individual Asset,NaN,NaN,NaN,NaN,NaN,XRP-USD,2019-01-02,2019-12-30,363 days,100.0,99.913462,-0.086538,-48.173852,0.474629,5.865632e-02,0.137819,191 days 00:00:00,94.0,94.0,0.0,0.000000,25.531915,28.723192,-9.132107,5.631823,-2.247448,3 days 07:00:00,1 days 04:48:00,0.831242,-9.206205e-04,-0.494885,-0.631372,0.887646,-0.808584
4,2019,Individual Asset,NaN,NaN,NaN,NaN,NaN,BCH-USD,2019-01-02,2019-12-30,363 days,100.0,158.252876,58.252876,21.869270,100.000000,2.658461e+01,48.872519,229 days 00:00:00,94.0,94.0,0.0,0.000000,35.106383,70.684594,-10.840778,8.003329,-2.991709,2 days 18:10:54.545454545,1 days 03:32:27.540983606,1.216681,6.197114e-01,0.941843,1.200135,1.277016,2.017615
5,2019,Individual Asset,NaN,NaN,NaN,NaN,NaN,EOS-USD,2019-01-02,2019-12-30,363 days,100.0,99.896994,-0.103006,-8.383521,8.270341,7.661991e-01,3.163920,216 days 00:00:00,97.0,97.0,0.0,0.000000,26.804124,35.625649,-15.128080,8.020575,-2.859502,3 days 02:46:09.230769230,1 days 08:47:19.436619718,0.987760,-1.061914e-03,-0.018081,-0.032736,0.995781,-0.027964
6,2019,Individual Asset,NaN,NaN,NaN,NaN,NaN,LTC-USD,2019-01-02,2019-12-30,363 days,100.0,172.487173,72.487173,27.857105,76.620953,1.185698e+01,12.226189,184 days 00:00:00,87.0,87.0,0.0,0.000000,39.080460,51.882708,-10.483983,7.750090,-2.613998,2 days 21:52:56.470588235,1 days 03:37:21.509433962,1.760458,8.331859e-01,1.756797,5.971283,1.522496,3.573080
7,2019,Individual Asset,NaN,NaN,NaN,NaN,NaN,XLM-USD,2019-01-02,2019-12-30,363 days,100.0,100.032263,0.032263,-61.490309,0.141434,1.540720e-02,0.035161,155 days 00:00:00,88.0,88.0,0.0,0.000000,29.545455,40.713323,-8.917515,7.808478,-2.431234,3 days 06:27:41.538461538,1 days 10:50:19.354838709,1.236395,3.666227e-04,0.545827,0.922627,1.133575,0.948756
8,2019,Individual Asset,NaN,NaN,NaN,NaN,NaN,ADA-USD,2019-01-02,2019-12-30,363 days,100.0,100.000749,0.000749,-25.772238,0.097697,1.006625e-02,0.029751,201 days 00:00:00,92.0,92.0,0.0,0.000000,29.347826,20.572796,-8.426702,6.538109,-2.783509,3 days 03:33:20,1 days 10:20:18.461538461,1.007241,8.139006e-06,0.018370,0.025308,1.004053,0.029499
9,2019,Individual Asset,NaN,NaN,NaN,NaN,NaN,TRX-USD,2019-01-02,2019-12-30,363 days,100.0,100.011442,0.011442,-33.735540,0.038551,4.017019e-03,0.010006,211 days 00:00:00,90.0,90.0,0.0,0.000000,34.444444,27.588414,-9.776020,6.412079,-2.494185,2 days 23:13:32.903225806,1 days 08:32:32.542372881,1.336793,1.271331e-04,0.690456,1.149840,1.163482,1.124167


In [1]:
"""
Cryptocurrency Trading Strategy with GPR+LSTM Model

This program implements and backtests a trading strategy for cryptocurrencies
using a combination of Gaussian Process Regression with changepoint detection
and LSTM neural networks.

Author: agehcx
Date: 2025-03-07
"""

import yfinance as yf
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from gpflow.kernels import Matern32, Kernel
from gpflow.models import GPR
from gpflow import set_trainable
from sklearn.preprocessing import StandardScaler
import vectorbt as vbt
from datetime import datetime
import ruptures as rpt
import warnings

# Suppress warnings for cleaner output
warnings.filterwarnings("ignore", category=FutureWarning)

#------------------------------------------------------------------------------
# CONFIGURATION
#------------------------------------------------------------------------------

# Model hyperparameters
MODEL_PARAMS = {
    'batch_size': 128,
    'dropout_rate': 0.2,
    'learning_rate': 0.001,
    'lookback_window': 21,
    'lstm_hidden_units': 40,
    'epochs': 50,
    'train_ratio': 0.8
}

# Trading parameters
TRADING_PARAMS = {
    'fee_rate': 0.001,  # 0.1% fee per trade
    'position_size': 1.0,  # Full position size
    'frequency': '1D'     # Daily rebalancing
}

# Dictionary of top cryptocurrencies by year with -USD suffix
TOP_CRYPTOS = {
    "2019": ["BTC-USD", "ETH-USD", "XRP-USD", "BCH-USD", "EOS-USD", "LTC-USD", "XLM-USD", "ADA-USD", "TRX-USD", "BSV-USD"],
    "2020": ["BTC-USD", "ETH-USD", "XRP-USD", "BCH-USD", "LTC-USD", "EOS-USD", "BNB-USD", "BSV-USD", "ADA-USD", "XTZ-USD"],
    "2021": ["BTC-USD", "ETH-USD", "XRP-USD", "LTC-USD", "BCH-USD", "ADA-USD", "DOT-USD", "LINK-USD", "BNB-USD", "XLM-USD"],
    "2022": ["BTC-USD", "ETH-USD", "BNB-USD", "SOL-USD", "ADA-USD", "XRP-USD", "DOT-USD", "LUNA-USD", "AVAX-USD", "DOGE-USD"],
    "2023": ["BTC-USD", "ETH-USD", "BNB-USD", "XRP-USD", "ADA-USD", "DOGE-USD", "MATIC-USD", "DOT-USD", "LTC-USD", "SHIB-USD"],
    "2024": ["BTC-USD", "ETH-USD", "BNB-USD", "XRP-USD", "ADA-USD", "DOGE-USD", "SOL-USD", "DOT-USD", "LTC-USD", "AVAX-USD"],
    "2025": ["BTC-USD", "ETH-USD", "BNB-USD", "XRP-USD", "ADA-USD", "DOGE-USD", "SOL-USD", "DOT-USD", "LTC-USD", "AVAX-USD"]
}

# Output file
RESULTS_FILE = "Backtested/equal_weight_portfolio_results_signal2.csv"

#------------------------------------------------------------------------------
# DATA PROCESSING
#------------------------------------------------------------------------------

def fetch_and_process_data(tickers, start_date, end_date):
    """
    Fetch data for multiple tickers and return price and standardized returns DataFrames.
    
    Args:
        tickers (list): List of ticker symbols to fetch data for
        start_date (str): Start date in YYYY-MM-DD format
        end_date (str): End date in YYYY-MM-DD format
        
    Returns:
        tuple: (prices_df, returns_df) - DataFrames containing price data and standardized returns
    """
    price_data = {}
    returns_data = {}
    
    for ticker in tickers:
        try:
            # Download data from Yahoo Finance
            data = yf.download(ticker, start=start_date, end=end_date)
            if data.empty:
                print(f"No data found for {ticker} between {start_date} and {end_date}")
                continue
                
            # Prefer "Adj Close" if available, otherwise "Close"
            if "Adj Close" in data.columns:
                price_series = data["Adj Close"].copy()
            else:
                price_series = data["Close"].copy()
                
            # Ensure we have a series, not a DataFrame
            if isinstance(price_series, pd.DataFrame):
                price_series = price_series.squeeze()
                
            price_series.name = ticker
            
            # Calculate returns and drop any NA values
            returns = price_series.pct_change().dropna()
            
            # Standardize returns for better model performance
            scaler = StandardScaler()
            std_returns = scaler.fit_transform(returns.values.reshape(-1, 1))
            valid_index = returns.index
            
            price_data[ticker] = price_series.loc[valid_index]
            returns_data[ticker] = pd.Series(
                std_returns.flatten(), index=valid_index, name=ticker
            )
            
        except Exception as e:
            print(f"Error fetching {ticker}: {e}")
    
    # Convert to DataFrames
    prices_df = pd.DataFrame({ticker: series for ticker, series in price_data.items()})
    returns_df = pd.DataFrame({ticker: series for ticker, series in returns_data.items()})
    
    return prices_df, returns_df

#------------------------------------------------------------------------------
# CHANGEPOINT DETECTION
#------------------------------------------------------------------------------

class ChangePointKernel(Kernel):
    """
    Custom kernel for Gaussian Process Regression that accounts for regime changes.
    
    This kernel multiplies a base kernel with a mask that's 1 when two points are in the
    same regime (separated by changepoints) and 0 otherwise.
    """
    def __init__(self, base_kernel, changepoints):
        """
        Initialize the ChangePointKernel.
        
        Args:
            base_kernel: The base kernel (e.g., Matern32)
            changepoints (list): Indices where regime changes are detected
        """
        super().__init__()
        self.base_kernel = base_kernel
        # changepoints: list of indices where a regime change is detected
        self.changepoints = changepoints

    def get_region(self, X):
        """
        Determine the region (between which changepoints) each data point falls.
        
        Args:
            X: Input tensor
            
        Returns:
            Tensor containing the region index for each input point
        """
        cp = tf.constant(self.changepoints, dtype=X.dtype)
        regions = tf.searchsorted(cp, X[:, 0], side='right')
        return regions

    def K(self, X, X2=None):
        """
        Compute covariance matrix between inputs X and X2.
        
        Args:
            X: First input
            X2: Second input (defaults to X if None)
            
        Returns:
            Covariance matrix
        """
        if X2 is None:
            X2 = X
        regions_X = self.get_region(X)
        regions_X2 = self.get_region(X2)
        # Create a mask that's 1 when points are in the same region, 0 otherwise
        regions_equal = tf.cast(tf.equal(tf.expand_dims(regions_X, 1), tf.expand_dims(regions_X2, 0)), X.dtype)
        # Get the base kernel's covariance matrix
        base_cov = self.base_kernel.K(X, X2)
        # Apply the mask to the base covariance
        return base_cov * regions_equal

    def K_diag(self, X):
        """
        Compute diagonal of covariance matrix for inputs X.
        
        Args:
            X: Input tensor
            
        Returns:
            Diagonal of covariance matrix
        """
        return self.base_kernel.K_diag(X)


def detect_changepoints(returns, lookback_window):
    """
    Detect changepoints in a return series using Pelt algorithm with RBF model.
    
    Args:
        returns (pd.Series): Series of returns to analyze
        lookback_window (int): Window size to consider for changepoint detection
        
    Returns:
        list: Indices where changepoints are detected
    """
    # Use ruptures to detect changepoints using an RBF model
    algo = rpt.Pelt(model="rbf").fit(returns.values)
    
    # The penalty parameter controls the number of changepoints
    # Using the lookback_window as penalty is a simplified approach
    result = algo.predict(pen=lookback_window)
    
    return result

#------------------------------------------------------------------------------
# MODEL IMPLEMENTATION
#------------------------------------------------------------------------------

def generate_signals(returns, changepoints, train_ratio, epochs, 
                     dropout_rate, lstm_hidden_units, learning_rate, batch_size):
    """
    Generate trading signals using combined GPR+LSTM model.
    
    Args:
        returns (pd.Series): Standardized returns to model
        changepoints (list): Detected changepoints in the series
        train_ratio (float): Ratio of data to use for training (0.0-1.0)
        epochs (int): Number of training epochs for LSTM
        dropout_rate (float): Dropout rate for LSTM
        lstm_hidden_units (int): Number of hidden units in LSTM layer
        learning_rate (float): Learning rate for Adam optimizer
        batch_size (int): Batch size for training
        
    Returns:
        numpy.ndarray: Array of position signals (-1 for short, 1 for long)
    """
    # Use time index as a feature
    X = np.arange(len(returns), dtype=np.float64).reshape(-1, 1)
    y = returns.values.reshape(-1, 1)
    
    # Use Gaussian Process Regression with the ChangePointKernel for trend extraction
    base_kernel = Matern32()
    kernel = ChangePointKernel(base_kernel, changepoints)
    gpr = GPR(data=(X, y), kernel=kernel)
    set_trainable(gpr.likelihood.variance, False)
    
    # Extract the trend component using GPR
    trend = gpr.predict_f(X)[0].numpy().flatten()
    
    # Combine returns and trend to form a feature set
    features = np.hstack([returns.values.reshape(-1, 1), trend.reshape(-1, 1)])
    split_index = int(len(features) * train_ratio)
    
    # Split into training and testing data
    train_features = features[:split_index]
    train_labels = returns.values[:split_index]
    
    # Reshape features for LSTM input (samples, timesteps, features)
    train_features = train_features.reshape((train_features.shape[0], 1, train_features.shape[1]))
    
    # Build the LSTM model
    model = Sequential([
        # First LSTM layer returns sequences for stacking
        LSTM(lstm_hidden_units, return_sequences=True,
             input_shape=(train_features.shape[1], train_features.shape[2])),
        Dropout(dropout_rate),  # Apply dropout to prevent overfitting
        # Second LSTM layer
        LSTM(int(lstm_hidden_units / 2)),
        # Output layer with tanh activation (outputs between -1 and 1)
        Dense(1, activation="tanh")
    ])
    
    # Compile the model with Adam optimizer and MSE loss
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), loss="mse")
    
    # Train the model
    model.fit(train_features, train_labels, epochs=epochs, batch_size=batch_size, verbose=0)
    
    # Predict signals for all features
    all_features = features.reshape((features.shape[0], 1, features.shape[1]))
    predicted_signals = model.predict(all_features, verbose=0).flatten()
    
    # Convert signals to binary (-1 or 1) for direction only
    # For equal weighting, we only care about direction (long/short)
    positions = np.sign(predicted_signals)
    positions[positions == 0] = 1  # Convert any zeros to 1 (long)
    
    return positions

#------------------------------------------------------------------------------
# BACKTESTING
#------------------------------------------------------------------------------

def backtest_strategy_single_asset(price_series, positions, fee_rate=0.001):
    """
    Backtest a trading strategy for a single asset.
    
    Args:
        price_series (pd.Series): Series of price data
        positions (pd.Series/array): Array of position signals (-1 for short, 1 for long)
        fee_rate (float): Fee rate as a decimal (default: 0.001 or 0.1%)
        
    Returns:
        vectorbt.Portfolio: Portfolio object containing backtest results
    """
    # Ensure price_series is a Series, not a DataFrame
    if isinstance(price_series, pd.DataFrame):
        price_series = price_series.squeeze()
    
    # Convert positions to a Series with the same index as price_series
    # positions = pd.Series(positions, index=price_series.index, name=price_series.name) ## Old 
    positions = pd.Series(positions, index=price_series.index[:len(positions)], name=price_series.name) ## New
    
    # Generate entry/exit signals from positions
    entries = positions > 0  # Long signals
    exits = positions < 0    # Short signals
    
    # Create portfolio using vectorbt
    pf = vbt.Portfolio.from_signals(
        close=price_series,
        entries=entries,
        exits=exits,
        # signal=positions2, # Use positions as signals instead of entries/exits
        size=1.0,  # Always use full position size
        freq="1D",  # Daily data
        fees=fee_rate,
    )
    return pf


def backtest_equal_weight_portfolio(prices_df, signals_dict, fee_rate=0.001):
    """
    Backtest a portfolio with equal weights across assets.
    
    Args:
        prices_df (pd.DataFrame): DataFrame of price data for multiple assets
        signals_dict (dict): Dictionary mapping asset names to position signals
        fee_rate (float): Fee rate for transactions
        
    Returns:
        tuple: (portfolio, asset_portfolios) - Portfolio object and dict of individual asset portfolios
    """
    # Create a dictionary to store individual asset performance
    asset_results = {}
    
    for ticker in prices_df.columns:
        if ticker in signals_dict:
            try:
                # Get price series and signals for this ticker
                price_series = prices_df[ticker]
                positions = signals_dict[ticker]
                
                # Make sure positions align with price index
                positions_series = pd.Series(positions, index=price_series.index[:len(positions)])
                
                # Backtest the individual asset
                pf = backtest_strategy_single_asset(price_series, positions_series, fee_rate)
                asset_results[ticker] = pf
            except Exception as e:
                print(f"Error backtesting {ticker}: {e}")
    
    # Calculate the equal weight allocation (1/n for each asset)
    n_assets = len(asset_results)
    if n_assets == 0:
        return None, {}
        
    weight_per_asset = 1.0 / n_assets
    
    # Create a combined DataFrame for returns
    combined_returns = pd.DataFrame()
    
    # Extract returns from each asset portfolio
    for ticker, pf in asset_results.items():
        returns = pf.returns()
        if not returns.empty:
            combined_returns[ticker] = returns
    
    # Calculate equal-weighted portfolio returns
    if not combined_returns.empty:
        # Replace NaN with 0 for calculation
        combined_returns.fillna(0, inplace=True)
        portfolio_returns = (combined_returns * weight_per_asset).sum(axis=1)
        
        # Calculate cumulative returns
        portfolio_cumulative = (1 + portfolio_returns).cumprod()
        
        # Create a mock portfolio object with key stats
        portfolio_stats = {
            "total_return": portfolio_cumulative.iloc[-1] - 1 if len(portfolio_cumulative) > 0 else 0,
            "sharpe_ratio": portfolio_returns.mean() / portfolio_returns.std() * np.sqrt(252) if len(portfolio_returns) > 0 else 0,
            "max_drawdown": (portfolio_cumulative / portfolio_cumulative.cummax() - 1).min() if len(portfolio_cumulative) > 0 else 0,
            "win_rate": np.sum(portfolio_returns > 0) / len(portfolio_returns) if len(portfolio_returns) > 0 else 0,
        }
        
        # Create mock portfolio object with stats method
        mock_portfolio = type('obj', (object,), {
            'stats': lambda: portfolio_stats
        })
        
        return mock_portfolio, asset_results
    else:
        return None, {}

#------------------------------------------------------------------------------
# MAIN FUNCTION
#------------------------------------------------------------------------------

def main():
    """
    Main function to execute the cryptocurrency trading strategy.
    
    This function:
    1. Defines the test periods
    2. Processes data for each period
    3. Generates signals using the GPR+LSTM model
    4. Backtests the equal-weight portfolio strategy
    5. Saves and displays the results
    """
    # Define periods (one per year)
    periods = []
    for year in range(2019, 2025):
        period_label = str(year)
        start_date = f"{year}-01-01"
        end_date = f"{year}-12-31"
        periods.append((period_label, start_date, end_date))
        
    # YTD 2025 using current date
    periods.append(("2025_YTD", "2025-01-01", "2025-03-13"))

    results = []

    # Loop over each period and backtest the respective top cryptocurrencies for that year
    for period_label, start_date, end_date in periods:
        year = period_label.split("_")[0]  # Extract year from period label
        cryptos_for_year = TOP_CRYPTOS.get(year, [])
        
        print(f"\nProcessing period {period_label} ({start_date} to {end_date})")
        print(f"Cryptocurrencies: {', '.join(cryptos_for_year)}")
        
        # Fetch all data for the period
        prices_df, returns_df = fetch_and_process_data(cryptos_for_year, start_date, end_date)
        
        # Generate signals for each cryptocurrency
        signals_dict = {}
        
        for crypto in cryptos_for_year:
            if crypto in returns_df.columns:
                try:
                    returns = returns_df[crypto]
                    if len(returns) < MODEL_PARAMS['lookback_window']:
                        print(f"Not enough data for {crypto} in period {period_label}. Skipping.")
                        continue
                    
                    # Detect changepoints in the return series
                    changepoints = detect_changepoints(returns, lookback_window=MODEL_PARAMS['lookback_window'])
                    
                    # Generate trading signals using the GPR+LSTM model
                    positions = generate_signals(
                        returns, changepoints,
                        train_ratio=MODEL_PARAMS['train_ratio'],
                        epochs=MODEL_PARAMS['epochs'],
                        dropout_rate=MODEL_PARAMS['dropout_rate'],
                        lstm_hidden_units=MODEL_PARAMS['lstm_hidden_units'],
                        learning_rate=MODEL_PARAMS['learning_rate'],
                        batch_size=MODEL_PARAMS['batch_size']
                    )
                    
                    signals_dict[crypto] = positions
                except Exception as e:
                    print(f"Error generating signals for {crypto}: {e}")
        
        # Backtest with equal weight allocation
        portfolio, asset_portfolios = backtest_equal_weight_portfolio(
            prices_df, signals_dict, fee_rate=TRADING_PARAMS['fee_rate']
        )
        
        if portfolio:
            # Record main portfolio results
            portfolio_stats = portfolio.stats()
            result = {
                "period": period_label,
                "strategy": "Equal Weight Portfolio",
                "num_assets": len(signals_dict),
                **{key: str(value) if pd.isna(value) else value for key, value in portfolio_stats.items()}
            }
            results.append(result)
            
            # Also record individual asset results
            for crypto, pf in asset_portfolios.items():
                asset_stats = pf.stats()
                asset_result = {
                    "period": period_label,
                    "crypto": crypto,
                    "strategy": "Individual Asset",
                    **{key: str(value) if pd.isna(value) else value for key, value in asset_stats.items()}
                }
                results.append(asset_result)

    # Save all summary results into a DataFrame
    results_df = pd.DataFrame(results)
    print("\nBacktest Results:")
    print(results_df)
    
    # Save the DataFrame to a CSV file
    results_df.to_csv(RESULTS_FILE, index=False)
    
    # Calculate and display yearly performance comparison
    portfolio_results = results_df[results_df["strategy"] == "Equal Weight Portfolio"]
    if not portfolio_results.empty:
        print("\nEqual Weight Portfolio Performance by Year:")
        yearly_perf = portfolio_results[["period", "total_return", "sharpe_ratio", "max_drawdown"]]
        print(yearly_perf)


if __name__ == "__main__":
    main()


Processing period 2019 (2019-01-01 to 2019-12-31)
Cryptocurrencies: BTC-USD, ETH-USD, XRP-USD, BCH-USD, EOS-USD, LTC-USD, XLM-USD, ADA-USD, TRX-USD, BSV-USD
YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
/opt/homebrew/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
[********************


Processing period 2020 (2020-01-01 to 2020-12-31)
Cryptocurrencies: BTC-USD, ETH-USD, XRP-USD, BCH-USD, LTC-USD, EOS-USD, BNB-USD, BSV-USD, ADA-USD, XTZ-USD



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
/opt/homebrew/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
[*********************100%***********************]  1 of 1 completed


Processing period 2021 (2021-01-01 to 2021-12-31)
Cryptocurrencies: BTC-USD, ETH-USD, XRP-USD, LTC-USD, BCH-USD, ADA-USD, DOT-USD, LINK-USD, BNB-USD, XLM-USD


...


Processing period 2024 (2024-01-01 to 2024-12-31)
Cryptocurrencies: BTC-USD, ETH-USD, BNB-USD, XRP-USD, ADA-USD, DOGE-USD, SOL-USD, DOT-USD, LTC-USD, AVAX-USD



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
/opt/homebrew/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
[*********************100%***********************]  1 of 1 completed
[*******************


Processing period 2025_YTD (2025-01-01 to 2025-03-13)
Cryptocurrencies: BTC-USD, ETH-USD, BNB-USD, XRP-USD, ADA-USD, DOGE-USD, SOL-USD, DOT-USD, LTC-USD, AVAX-USD



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
/opt/homebrew/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)



Backtest Results:
      period                strategy  num_assets  total_return  sharpe_ratio  \
0       2019  Equal Weight Portfolio        10.0      0.437054      1.376499   
1       2019        Individual Asset         NaN           NaN           NaN   
2       2019        Individual Asset         NaN           NaN           NaN   
3       2019        Individual Asset         NaN           NaN           NaN   
4       2019        Individual Asset         NaN           NaN           NaN   
..       ...                     ...         ...           ...           ...   
72  2025_YTD        Individual Asset         NaN           NaN           NaN   
73  2025_YTD        Individual Asset         NaN           NaN           NaN   
74  2025_YTD        Individual Asset         NaN           NaN           NaN   
75  2025_YTD        Individual Asset         NaN           NaN           NaN   
76  2025_YTD        Individual Asset         NaN           NaN           NaN   

    max_drawdown  wi